### Hitta toaletter längs Roslagsleden med OSM
* denna notebook
* Wikidata Roslagsleden [Q10657105](https://www.wikidata.org/wiki/Q10657105?uselang=sv)
* Issue [#4](https://github.com/salgo60/Roslagsleden/issues/4)

----
* test 1 [Issue 4 toaletter_nara_roslagsleden 2025_07_28_13_41](https://raw.githack.com/salgo60/Roslagsleden/main/kartor/Issue%204%20toaletter_nara_roslagsleden%202025_07_28_13_41.html)
* test 2 [500 meter](https://raw.githack.com/salgo60/Roslagsleden/main/kartor/Issue%204%20500%20meter%20toaletter_nara_roslagsleden%202025_07_28_17_57.html)
 
* test 3 [2000 meter](https://raw.githack.com/salgo60/Roslagsleden/main/kartor/Issue%204%202000%20meter%20toaletter_nara_roslagsleden%202025_07_28_17_58.html)


In [1]:
import time
from datetime import datetime

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))


Start: 2025-07-28 13:24:27


In [8]:
# Toaletter inom 200 meter fran Roslagsleden (via Wikidata P402)

# Installera beroenden om de saknas
try:
    import geopandas as gpd
    import shapely
    import folium
    import requests
    from SPARQLWrapper import SPARQLWrapper, JSON
except ImportError:
    import sys
    !{sys.executable} -m pip install geopandas shapely folium requests SPARQLWrapper

import requests
import geopandas as gpd
from shapely.geometry import LineString, Point
from shapely.ops import unary_union
import folium
from SPARQLWrapper import SPARQLWrapper, JSON

# Steg 1: Hämta OSM relationer via Wikidata (Roslagsleden = Q10657105)
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""
SELECT ?osmid WHERE {
  ?item wdt:P361 wd:Q10657105;
        wdt:P31 wd:Q2143825;
        wdt:P402 ?osmid.
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

osm_ids = [result['osmid']['value'] for result in results['results']['bindings']]
print(f"Hämtade {len(osm_ids)} OSM-relationer från Wikidata")

# Steg 2: Hämta geometrier från Overpass
features = []
overpass_url = "http://overpass-api.de/api/interpreter"
for rel_id in osm_ids:
    overpass_query = f"""
    [out:json];
    relation({rel_id});
    (._;>>;);
    out geom;
    """
    response = requests.post(overpass_url, data={'data': overpass_query})
    if response.status_code != 200:
        print(f"Fel vid hämtning av relation {rel_id}: {response.text}")
        continue
    data = response.json()
    for element in data['elements']:
        if element['type'] == 'way' and 'geometry' in element:
            coords = [(pt['lon'], pt['lat']) for pt in element['geometry']]
            features.append(LineString(coords))

# Kontroll: Avbryt om inga geometrier hittades
if not features:
    raise ValueError("Inga geometrier hittades från OSM-relationer kopplade via Wikidata.")

# Kombinera alla ways till en enda linje (MultiLine)
trail_geom = unary_union(features)
gdf_trail = gpd.GeoDataFrame(geometry=[trail_geom], crs="EPSG:4326")

# Kontroll: Avbryt om geometrin är tom
if gdf_trail.is_empty.any():
    raise ValueError("Geometrin är tom. Kan inte fortsätta.")

# Steg 3: Buffra 200 meter
trail_utm = gdf_trail.to_crs(epsg=3006)  # SWEREF99 TM
buffered_utm = trail_utm.buffer(200)
buffered = buffered_utm.to_crs(epsg=4326)

# Steg 4: Hämta toaletter via Overpass inom bounding box
bbox = gdf_trail.total_bounds  # minx, miny, maxx, maxy
print("Bounding box:", bbox)
query_toilets = f"""
[out:json][timeout:25];
node["amenity"="toilets"]({bbox[1]},{bbox[0]},{bbox[3]},{bbox[2]});
out;
"""
toilets_response = requests.post(overpass_url, data={'data': query_toilets})

if toilets_response.status_code != 200:
    print("Fel vid Overpass-frågan:")
    print(toilets_response.text)
    toilets_response.raise_for_status()

try:
    toilets_data = toilets_response.json()
except Exception as e:
    print("Kunde inte tolka JSON-svar från Overpass:")
    print(toilets_response.text)
    raise e

# Skapa GeoDataFrame av toaletter
toilet_points = []
for el in toilets_data['elements']:
    if el['type'] == 'node':
        pt = Point(el['lon'], el['lat'])
        tags = el.get('tags', {})
        toilet_points.append({"geometry": pt, "tags": tags})

gdf_toilets = gpd.GeoDataFrame(toilet_points, crs="EPSG:4326")

# Steg 5: Filtrera de som ligger inom bufferten
in_range = gdf_toilets[gdf_toilets.geometry.within(buffered.geometry.unary_union)]

# Steg 6: Visualisera med Folium
m = folium.Map(location=[gdf_trail.geometry[0].centroid.y, gdf_trail.geometry[0].centroid.x], zoom_start=10)
folium.GeoJson(gdf_trail.geometry[0], name="Roslagsleden").add_to(m)
folium.GeoJson(buffered.geometry[0], name="200m Buffert").add_to(m)

for idx, row in in_range.iterrows():
    tags = row['tags']
    el = toilets_data['elements'][idx]
    osm_url = f"https://www.openstreetmap.org/node/{el['id']}"

    # Lista viktiga taggar
    required_tags = ["amenity", "access", "unisex", "toilets:disposal", "wheelchair"]
    recommended_tags = ["toilets:handwashing", "toilets:paper_supply", "opening_hours", "fee", "operator", "description"]
    extra_tags = ["image", "contact:phone", "contact:email", "website", "fixme", "check_date"]
    problem_tags = {"access": "private", "key": "yes", "fee": "yes"}

    missing = [t for t in required_tags if t not in tags]
    warnings = [f"⚠️ {k}={v} bör undvikas" for k, v in problem_tags.items() if tags.get(k) == v]
    recommended_missing = [f"➕ {t} saknas" for t in recommended_tags if t not in tags]

    suggestions = "<br>".join(warnings + recommended_missing + ["<i>Tips: kontrollera viktiga attribut</i>"])
    tag_info = "<br>".join([f"{k}: {v}" for k, v in tags.items()])
    popup_text = f"<b><a href='{osm_url}' target='_blank'>OSM objekt</a></b><br>{tag_info}<br><br>{suggestions}"

    icon_type = "info-sign" if not warnings else "exclamation-sign"
    icon_color = 'red' if warnings else 'blue'
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=popup_text,
        icon=folium.Icon(icon=icon_type, color=icon_color)
    ).add_to(m)

from datetime import datetime

# Skapa filnamn med tidsstämpel
now = datetime.now()
timestamp = now.strftime("%Y_%m_%d_%H_%M")
output_path = f"../kartor/Issue 4 toaletter_nara_roslagsleden {timestamp}.html"
m.save(output_path)

print(f"Hittade {len(in_range)} toaletter inom 200 meter från Roslagsleden. Kartan sparades som '{output_path}'.")


Hämtade 11 OSM-relationer från Wikidata
Bounding box: [17.985927  59.4115678 18.8795578 60.0975035]
Hittade 15 toaletter inom 200 meter från Roslagsleden. Kartan sparades som '../kartor/Issue 4 toaletter_nara_roslagsleden 2025_07_28_13_41.html'.


/var/folders/fd/md6r13sj0wsbg_6_xl160d300000gn/T/ipykernel_8372/1960552341.py:106: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  in_range = gdf_toilets[gdf_toilets.geometry.within(buffered.geometry.unary_union)]


In [ ]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))